# Tidy Data in Python
by [Jean-Nicholas Hould](http://www.jeannicholashould.com/)

In [3]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re

## Column headers are values, not variable names

### Pew Research Center Dataset

In [6]:
df = pd.read_csv("./data/pew-raw.csv")
df.head(2)

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70


In [7]:
formatted_df = pd.melt(df,["religion"], 
                       var_name="income", value_name="freq")
formatted_df = formatted_df.sort_values(by=["religion"])
formatted_df.head(2)

,religion,income,freq
0,Agnostic,<$10k,27
30,Agnostic,$30-40k,81


### Billboard Top 100 Dataset

In [42]:
df = pd.read_csv("./data/billboard.csv", encoding="mac_latin2")
df.head(2)

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# Melting
id_vars = ["year","artist.inverted","track","time","genre","date.entered","date.peaked"]
df = pd.melt(frame=df,id_vars=id_vars, var_name="week", value_name="rank")
df[:3]

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,x1st.week,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,x1st.week,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,x1st.week,71.0


In [44]:
# Formatting 
df["week"] = df['week'].str.extract('(\d+)', expand=False).astype(int)

In [45]:
df[-3:]

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
24089,2000,Zombie Nation,Kernkraft 400,3:30,Rock,2000-09-02,2000-09-02,76,NaN
24090,2000,"Eastsidaz, The",Got Beef,3:58,Rap,2000-07-01,2000-07-01,76,NaN
24091,2000,Fragma,Toca's Miracle,3:22,R&B,2000-10-28,2000-10-28,76,NaN


In [46]:
# Cleaning out unnecessary rows
df = df.dropna()
df["rank"] = df["rank"].astype(int)
df[-3:]

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
19980,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,64,50
20017,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,64,50
20334,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,65,49


In [47]:
# Create "date" columns
df['date'] = pd.to_datetime(df['date.entered']) + pd.to_timedelta(df['week'], unit='w') - pd.DateOffset(weeks=1)

df = df[["year", "artist.inverted", "track", "time", "genre", "week", "rank", "date"]]
df = df.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])

# Assigning the tidy dataset to a variable for future usage
billboard = df

df.head(2)

,year,artist.inverted,track,time,genre,week,rank,date
246,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,1,87,2000-02-26
563,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,2,82,2000-03-04


## Multiple types in one table

In [18]:
songs_cols = ["year", "artist.inverted", "track", "time", "genre"]
songs = billboard[songs_cols].drop_duplicates()
songs = songs.reset_index(drop=True)
songs["song_id"] = songs.index
songs.head(2)

,year,artist.inverted,track,time,genre,song_id
0,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,0
1,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,1


In [20]:
ranks = pd.merge(billboard, songs, on=["year","artist.inverted", "track", "time", "genre"])
ranks = ranks[["song_id", "date","rank"]]
ranks.head(2)

,song_id,date,rank
0,0,2000-02-26,87
1,0,2000-03-04,82


## Multiple variables stored in one column

### Tubercolosis Example

A few notes on the raw data set:

- The columns starting with "m" or "f" contain multiple variables: 
    - Sex ("m" or "f")
    - Age Group ("0-14","15-24", "25-34", "45-54", "55-64", "65", "unknown")
- Mixture of 0s and missing values("NaN"). This is due to the data collection process and the distinction is important for this dataset.

In [29]:
df = pd.read_csv("./data/tb-raw.csv")
df.head(2)

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0


In [30]:
df = pd.melt(df, id_vars=["country","year"], value_name="cases", var_name="sex_and_age")
df[:2]

,country,year,sex_and_age,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0


In [31]:
# Extract Sex, Age lower bound and Age upper bound group
tmp_df = df["sex_and_age"].str.extract("(\D)(\d+)(\d{2})", expand=False)    
tmp_df[:2]

,0,1,2
0,m,0,14
1,m,0,14


In [32]:
# Name columns
tmp_df.columns = ["sex", "age_lower", "age_upper"]
tmp_df[:2]

,sex,age_lower,age_upper
0,m,0,14
1,m,0,14


In [33]:
# Create `age`column based on `age_lower` and `age_upper`
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]
tmp_df[:2]

,sex,age_lower,age_upper,age
0,m,0,14,0-14
1,m,0,14,0-14


In [34]:
# Merge 
df = pd.concat([df, tmp_df], axis=1)
df[:2]

,country,year,sex_and_age,cases,sex,age_lower,age_upper,age
0,AD,2000,m014,0.0,m,0,14,0-14
1,AE,2000,m014,2.0,m,0,14,0-14


In [35]:
# Drop unnecessary columns and rows
df = df.drop(['sex_and_age',"age_lower","age_upper"], axis=1)
df = df.dropna()
df = df.sort_values(ascending=True,by=["country", "year", "sex", "age"])
df.head(2)

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
10,AD,2000,0.0,m,15-24


## Variables are stored in both rows and columns

### Global Historical Climatology Network Dataset

In [36]:
df = pd.read_csv("./data/weather-raw.csv")

In [37]:
df = pd.melt(df, id_vars=["id", "year","month","element"], var_name="day_raw")
df.head(4)

,id,year,month,element,day_raw,value
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN


In [38]:
# Extracting day
df["day"] = df["day_raw"].str.extract("d(\d+)", expand=False)  
df["id"] = "MX17004"
df[:2]

,id,year,month,element,day_raw,value,day
0,MX17004,2010,1,tmax,d1,NaN,1
1,MX17004,2010,1,tmin,d1,NaN,1


In [39]:
# To numeric values
df[["year","month","day"]] = df[["year","month","day"]].apply(lambda x: pd.to_numeric(x, errors='ignore'))

# Creating a date from the different columns
def create_date_from_year_month_day(row):
    return datetime.datetime(year=row["year"], month=int(row["month"]), day=row["day"])

df["date"] = df.apply(lambda row: create_date_from_year_month_day(row), axis=1)
df = df.drop(['year',"month","day", "day_raw"], axis=1)
df = df.dropna()
df[:2]

,id,element,value,date
12,MX17004,tmax,27.3,2010-02-02
13,MX17004,tmin,14.4,2010-02-02


In [40]:
# Unmelting column "element"
df = df.pivot_table(index=["id","date"], columns="element", values="value")
df.reset_index(drop=False, inplace=True)
df

element,id,date,tmax,tmin
0,MX17004,2010-02-02,27.3,14.4
1,MX17004,2010-02-03,24.1,14.4
2,MX17004,2010-03-05,32.1,14.2


## One type in multiple tables

### Baby Names in Illinois

In [41]:
def extract_year(string):
    match = re.match(".+(\d{4})", string) 
    if match != None: return match.group(1)
    
path = './data'
allFiles = glob.glob(path + "/201*-baby-names-illinois.csv")
df_list= []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    df.columns = map(str.lower, df.columns)
    df["year"] = extract_year(file_)
    df_list.append(df)
    
df = pd.concat(df_list)
df.head(5)

,rank,name,frequency,sex,year
0,1,Noah,863,Male,2015
1,2,Liam,709,Male,2015
2,3,Alexander,703,Male,2015
3,4,Jacob,650,Male,2015
4,5,William,618,Male,2015
